# Google Cloud Vision Example

First make sure you set up your google account with a [Cloud Vision Project](https://cloud.google.com/vision/docs/getting-started##set_up_your_project).  Then you need to enter billing details (to verify you are human) and set up credentials.  This will download a `.json` file.

Before you work in this notebook, you have to do this to let python know about your [application default credentials](https://cloud.google.com/vision/docs/getting-started#adc).  So put the `.json` file in this folder (but do not commit it into git!).  Then `export GOOGLE_APPLICATION_CREDENTIALS=<path to your json file>`.

This code is based on [Google's face detection tutorial](https://cloud.google.com/vision/docs/face-tutorial).

## Requirements
```
google-api-python-client==1.5.0
Pillow==3.1.1
```

In [9]:
from collections import OrderedDict
from googleapiclient import discovery
import httplib2
from oauth2client.client import GoogleCredentials
import base64
import os
from PIL import Image
from PIL import ImageDraw
from IPython import display
from pymongo import MongoClient

In [ ]:
#Initialize Database
client = MongoClient()
db = client.download
coll = db.photos
photo = { "photo_name": "http:////",
         "faces": "test"
}

In [10]:
# Set up a connection to the Google service
DISCOVERY_URL='https://{api}.googleapis.com/$discovery/rest?version={apiVersion}'
credentials = GoogleCredentials.get_application_default()
service = discovery.build('vision', 'v1', credentials=credentials, discoveryServiceUrl=DISCOVERY_URL)

In [16]:
# load the input images
img_path="/home/resonon/tutorial/faces/random10"
batch_request = []
image_pathnames = []
# create a dictionary with all the image_pathname
images = [os.path.join(img_path, name) for name in os.listdir(img_path) ]

# get google vision results for batch 
for image_pathname in images:
    image = open(image_pathname,'rb')
    image_content = image.read()
    # fire off request for face detection from Google
    json_request = {
        'image': {'content': base64.b64encode(image_content)},
        'features': [{ 'type': 'FACE_DETECTION','maxResults': 4, }]}
    batch_request.append(json_request)
    image_pathnames.append(image_pathname)
    request = service.images().annotate(body={'requests': batch_request,})
    response = request.execute()
print(len(image_pathnames))

print('Found %s face%s' % (len(response['responses']), 
                           '' if len(response['responses']) == 1 else 's'))

10
Found 10 faces


In [17]:
def highlight_crop_faces(image_name, face):
    """Draws a polygon around the faces, then saves to output_filename.


Args:
      image: a file containing the image with the faces.
      faces: a list of faces found in the file. This should be in the format
          returned by the Vision API.
      output_filename: the name of the image file to be created, where the faces
          have polygons drawn around them.
    """
    print(1)
    im = Image.open(image_name)
    im_copy = im.copy()
    draw = ImageDraw.Draw(im_copy)
    
    home_path, filename = '/'.join(str(image).split('/')[:-2]),  str(image).split('/')[-1]
    cropped_dir = '/'.join(str(image).split('/')[-2:-1])+"_cropped"
    highligthed_dir = '/'.join(str(image).split('/')[-2:-1])+"_highlighted"

    cropped_pathname = []
    if face.has_key('faceAnnotations'):
        for i in range(len(face['faceAnnotations'])):
            vertices = face['faceAnnotations'][i]['fdBoundingPoly']['vertices']
            #print(vertices)
            box = [(v['x'], v['y']) for v in vertices if v.has_key('x') and v.has_key('y')]

            prefix = ''
            if len(face['faceAnnotations']) > 1:
                prefix=str(i)
                print(i)
            if len(box) == 4:
                cropped_pathname = os.path.join(home_path, cropped_dir, prefix + filename)
                cropped_pathnames.append(os.path.join(home_path, cropped_dir, prefix + filename))
                cropped = im.crop((box[0][0], box[1][1],box[2][0], box[2][1])) 
                cropped.save(cropped_pathname, 'jpeg')

                highligthed_pathname = os.path.join(home_path, highligthed_dir, prefix + filename)
                draw.line(box + [box[0]], width=5, fill='#00ff00')                
                im_copy.save(highligthed_pathname, 'jpeg')
            else:
                print(face['faceAnnotations'][i])
        #    im.save(highligthed_pathname, 'jpeg')

    del draw
    del im_copy
    return(cropped_pathname)

In [18]:
cropped_pathnames = []
for image, face in zip(image_pathnames, response['responses']):
    print("."),
    cropped_pathnames.append(highlight_crop_faces(image, face))

. 1
. 1
. 1
0
1
2
3
. 1
0
1
2
3
. 1
. 1
. 1
. 1
. 1
. 1


In [ ]:
for i in range(len(response['responses'])):
    if response['responses'][i].has_key('faceAnnotations'):
        print len(response['responses'][i]['faceAnnotations']),#[0]['fdBoundingPoly']['vertices']
#response['responses']['faceAnnotations'][0]['fdBoundingPoly']['vertices']
#response['responses']['faceAnnotations'][0]['fdBoundingPoly']['vertices']
#response['responses']['faceAnnotations'][0]['fdBoundingPoly']['vertices']
#response['responses']['faceAnnotations'][0]['fdBoundingPoly']['vertices']
#response['responses']['faceAnnotations'][0]['fdBoundingPoly']['vertices']


In [ ]:
#for image_pathname in images:
#    highlight_crop_faces(image_pathname, face_filename, result['faceAnnotations'], );
#    for annotation in result['faceAnnotations']:
#       for emotion in ['joy','sorrow','surprise','anger']:
#            print "%s: %s" % (emotion, annotation[emotion+'Likelihood'])

In [ ]:
c = 0
for result in response['responses']:
    if not result.has_key("faceAnnotations"):
        print(c, "no face")
    else:
        print(c),
        c += 1

In [ ]:
#cropped_pathnames = []
#for image, face, candidate in zip(image_pathnames, response['responses']):
#    print(image, face, candidate)
#    cropped_pathnames.append(highlight_crop_faces(image, face))

In [ ]:


#highlight_crop_faces(image_pathnames[0], response['responses'][0], candidates[0])

In [ ]:
# print out emotions for each one
likelyhood_to_number = {"UNKNOWN":-1,
"VERY_UNLIKELY":1,
"UNLIKELY":2,
"POSSIBLE":3,
"LIKELY":4,
"VERY_LIKELY":5}
for result, candidate, image, cropped_image in zip(response['responses'], candidates, image_pathnames, cropped_pathnames):
    print(candidate)
    display.display(display.Image(filename=image,  width=100), display.Image(filename=cropped_image,  width=100))
    if result.has_key('faceAnnotations'):
        for faceAnnotations in result['faceAnnotations']:
            emotions = []
            for emotion, smiley in zip(['joy', 'sorrow', 'surprise', 'anger'], ["😃", "😢", "😮", "😡"]):
                 emotions.append(emotion + ":" + str(likelyhood_to_number[faceAnnotations[emotion+"Likelihood"]]))
            print(" ".join(emotions))
    else:
        print("Image processing error!")
    print("")

        

In [ ]:
ann = response['responses'][6]['faceAnnotations']

In [ ]:
r = response['responses'][7]
r.has_key('faceAnnotations')

In [ ]:
(len(response['responses']), len(candidates), len(image_pathnames), len(cropped_pathnames))

In [ ]:
response['responses'][6]['faceAnnotations'][0]['fdBoundingPoly']['vertices']

Ressources

http://scikit-learn.org/stable/auto_examples/applications/face_recognition.html

http://scikit-learn.org/stable/datasets/labeled_faces.html

http://gaelvaroquaux.github.io/scikit-learn-tutorial/putting_together.html
